In [14]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [15]:
# Load the data
data = pd.read_csv("train.csv", low_memory=False)
data.drop_duplicates() # remove duplicates

,Characteristics.LotFeatures,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.features_reso.results,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,...,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.Heating,Structure.LivingArea,Structure.NewConstructionYN,Structure.ParkingFeatures,Structure.Rooms.RoomsTotal,Structure.YearBuilt,Tax.Zoning,UnitTypes.UnitTypeType
0,NaN,3200.0,NaN,NaN,NaN,NaN,NaN,"['Cooling.CeilingFans', 'Flooring.Carpet', 'Fl...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,0.0,commr,NaN
1,NaN,NaN,NaN,3.8,3.7,3.8,3.8,"['Appliances.GasRange', 'Appliances.Range', 'A...",NaN,4.2,...,NaN,NaN,['natural gas'],3175.0,False,['off alley'],6.0,NaN,NaN,NaN
2,NaN,18750.0,NaN,NaN,NaN,NaN,NaN,"['Flooring.Carpet', 'ParkingFeatures.Garage', ...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,1926.0,commr,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['CommunityFeatures.Lake', 'WaterfrontFeatures...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,agric,NaN
4,"['horses allowed', 'paddock', 'pond(s)']",NaN,NaN,3.6,NaN,NaN,3.6,"['Appliances.Refrigerator', 'Appliances.Microw...",NaN,4.2,...,NaN,NaN,['other'],0.0,False,"['unassigned', 'off street']",4.0,2006.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107432,NaN,NaN,NaN,3.8,NaN,NaN,3.8,"['Levels.One', 'ParkingFeatures.Attached', 'Pa...",NaN,4.1,...,NaN,2.0,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
107433,NaN,NaN,2.8,NaN,3.6,3.1,3.1,"['Appliances.Dishwasher', 'Appliances.Stainles...",3.2,NaN,...,0.0,NaN,"['natural gas', 'forced air']",0.0,False,"['assigned', 'off street']",9.0,1963.0,NaN,NaN
107434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['ExteriorFeatures.PrivateYard', 'ExteriorFeat...",NaN,NaN,...,NaN,2.5,NaN,NaN,False,['driveway'],NaN,NaN,NaN,NaN
107435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['ParkingFeatures.ParkingLot'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# DATA CLEANING OF STORIES

data.dropna(subset=['ImageData.style.stories.summary.label'], inplace=True)

# Map the 'ImageData.style.stories.summary.label' column to numeric values
story_mapping = {
    '1_story': 1,
    '1.5_stories': 1.5,
    '2_stories': 2,
    '2.5_stories': 2.5,
    '3_stories_or_more': 3
}
data['ImageData.style.stories.summary.label'] = data['ImageData.style.stories.summary.label'].map(story_mapping).fillna(0)


In [17]:
columnes_numeriques = data.select_dtypes(include=['number'])  # Selecciona columnes numèriques
columnes_categoriques = data.select_dtypes(exclude=['number'])  # Selecciona columnes no numèriques (categòriques)

In [18]:
columnes_numeriques = columnes_numeriques.apply(lambda col: col.fillna(col.mean(skipna=True)) if col.dtype in ['float64', 'int64'] else col)
columnes_numeriques

,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,ImageData.q1q6.summary.interior,ImageData.q1q6.summary.kitchen,...,Structure.BathroomsFull,Structure.BathroomsHalf,Structure.BedroomsTotal,Structure.BelowGradeFinishedArea,Structure.BelowGradeUnfinishedArea,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.LivingArea,Structure.Rooms.RoomsTotal,Structure.YearBuilt
2,18750.000000,3.132245,3.295914,3.241381,3.144462,3.206947,3.322328,3.496782,3.142627,3.224893,...,1.981484,0.493744,3.288566,713.618752,653.674539,0.978887,2.020401,1875.207666,7.53296,1926.000000
4,50147.800808,3.132245,3.600000,3.241381,3.144462,3.600000,3.322328,4.200000,3.142627,3.224893,...,1.000000,0.000000,2.000000,713.618752,653.674539,0.978887,2.020401,0.000000,4.00000,2006.000000
6,50147.800808,3.132245,3.000000,3.241381,3.144462,3.000000,3.322328,3.000000,3.142627,3.224893,...,7.000000,2.000000,5.000000,4384.000000,653.674539,2.000000,5.000000,787.000000,10.00000,1990.000000
17,218275.000000,3.132245,3.295914,3.241381,3.144462,3.206947,3.322328,3.496782,3.142627,3.224893,...,1.981484,0.493744,3.288566,713.618752,653.674539,0.978887,2.020401,1875.207666,7.53296,1967.000000
23,50147.800808,3.132245,3.900000,3.241381,3.144462,3.900000,3.322328,4.300000,3.142627,3.224893,...,2.000000,1.000000,3.000000,713.618752,653.674539,0.978887,2.020401,1282.000000,5.00000,1958.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107430,50147.800808,3.132245,3.000000,3.241381,3.144462,3.000000,3.322328,3.000000,3.142627,3.224893,...,3.000000,1.000000,3.288566,713.618752,653.674539,0.978887,2.020401,1875.207666,7.53296,1968.923965
107431,50147.800808,3.132245,2.000000,3.241381,3.144462,2.000000,3.322328,3.300000,3.142627,3.224893,...,2.000000,0.000000,2.000000,713.618752,653.674539,0.978887,2.000000,1875.207666,7.53296,1968.923965
107432,50147.800808,3.132245,3.800000,3.241381,3.144462,3.800000,3.322328,4.100000,3.142627,3.224893,...,1.000000,0.000000,2.000000,713.618752,653.674539,0.978887,2.000000,1875.207666,7.53296,1968.923965
107434,50147.800808,3.132245,3.295914,3.241381,3.144462,3.206947,3.322328,3.496782,3.142627,3.224893,...,3.000000,1.000000,5.000000,713.618752,653.674539,0.978887,2.500000,1875.207666,7.53296,1968.923965


In [19]:
# normalize all numeric columns
scaler = MinMaxScaler()
columnes_numeriques = pd.DataFrame(scaler.fit_transform(columnes_numeriques), columns=columnes_numeriques.columns)

In [20]:
columnes_numeriques.head()

,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,ImageData.q1q6.summary.interior,ImageData.q1q6.summary.kitchen,...,Structure.BathroomsFull,Structure.BathroomsHalf,Structure.BedroomsTotal,Structure.BelowGradeFinishedArea,Structure.BelowGradeUnfinishedArea,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.LivingArea,Structure.Rooms.RoomsTotal,Structure.YearBuilt
0,0.002560,0.426449,0.459183,0.448276,0.428892,0.441389,0.566421,0.567450,0.522592,0.570485,...,0.026420,0.054860,0.182698,0.091701,0.133512,0.046614,0.002577,0.036483,0.076091,0.951581
1,0.006846,0.426449,0.520000,0.448276,0.428892,0.520000,0.566421,0.727273,0.522592,0.570485,...,0.013333,0.000000,0.111111,0.091701,0.133512,0.046614,0.002577,0.000000,0.040404,0.991107
2,0.006846,0.426449,0.400000,0.448276,0.428892,0.400000,0.566421,0.454545,0.522592,0.570485,...,0.093333,0.222222,0.277778,0.563351,0.133512,0.095238,0.006378,0.015311,0.101010,0.983202
3,0.029798,0.426449,0.459183,0.448276,0.428892,0.441389,0.566421,0.567450,0.522592,0.570485,...,0.026420,0.054860,0.182698,0.091701,0.133512,0.046614,0.002577,0.036483,0.076091,0.971838
4,0.006846,0.426449,0.580000,0.448276,0.428892,0.580000,0.566421,0.750000,0.522592,0.570485,...,0.026667,0.111111,0.166667,0.091701,0.133512,0.046614,0.002577,0.024942,0.050505,0.967391


In [21]:
columnes_categoriques.head()

,Characteristics.LotFeatures,ImageData.features_reso.results,ImageData.room_type_reso.results,ImageData.style.exterior.summary.label,Listing.Dates.CloseDate,Listing.ListingId,Location.Address.CensusBlock,Location.Address.CensusTract,Location.Address.City,Location.Address.CountyOrParish,...,Location.Area.SubdivisionName,Location.School.HighSchoolDistrict,Property.PropertyType,Structure.Basement,Structure.Cooling,Structure.Heating,Structure.NewConstructionYN,Structure.ParkingFeatures,Tax.Zoning,UnitTypes.UnitTypeType
2,NaN,"['Flooring.Carpet', 'ParkingFeatures.Garage', ...","['Community', 'FrontOfStructure', 'Other', 'Ga...",bungalow,2023-12-04T00:00:00,mrd09595738,17-31-210900-2014,17-31-210900,chicago,cook,...,NaN,NaN,commercial sale,NaN,['central air'],NaN,False,NaN,commr,NaN
4,"['horses allowed', 'paddock', 'pond(s)']","['Appliances.Refrigerator', 'Appliances.Microw...","['FrontOfStructure', 'BackOfStructure', 'Commu...",ranch,2023-09-20T00:00:00,mrd09795957,17-89-852403-1029,17-89-852403,maple park,kane,...,NaN,301,residential,['full'],"['partial', 'none']",['other'],False,"['unassigned', 'off street']",NaN,NaN
6,"['cul-de-sac', 'fenced yard', 'landscaped', 'w...",['Levels.Two'],['FrontOfStructure'],art_deco,2024-03-08T00:00:00,mrd10143654,17-97-864700-3009,17-97-864700,highland park,lake,...,NaN,113,residential,['full'],"['central air', 'zoned']",['natural gas'],False,NaN,NaN,NaN
17,NaN,['Levels.OneAndOneHalf'],"['FloorPlan', 'Map', 'FrontOfStructure']",cape_cod,2024-04-26T00:00:00,mrd10600330,NaN,NaN,lockport,will,...,NaN,NaN,commercial sale,NaN,['central air'],NaN,NaN,NaN,other,NaN
23,NaN,['Levels.One'],['FrontOfStructure'],bungalow,2024-04-18T00:00:00,mrd10641258,17-31-807600-4032,17-31-807600,skokie,cook,...,NaN,NaN,residential,['partial'],['none'],['natural gas'],False,NaN,NaN,NaN


In [22]:
columnes_numeriques.to_csv('columnes_numeriques.csv', index=False)
columnes_categoriques.to_csv('columnes_categoriques.csv', index=False)